### Multiplicação de matrizes com numpy


In [303]:
import pandas as pd
import glob
import os

import locale

# Defina o locale para português do Brasil
locale.setlocale(locale.LC_TIME, 'Portuguese_Brazil.1252')

'Portuguese_Brazil.1252'

In [304]:
caminho_pasta = r"C:\Users\gpira\Downloads\DADOS_COVI\DADOS_COVI"

In [305]:
print(f"📂 Buscando arquivos na pasta: {caminho_pasta}")

📂 Buscando arquivos na pasta: C:\Users\gpira\Downloads\DADOS_COVI\DADOS_COVI


In [333]:
# Busca todos os arquivos .csv dentro da pasta
padrao_busca = os.path.join(caminho_pasta, "*0.csv")
lista_arquivos = glob.glob(padrao_busca)
lista_arquivos

['C:\\Users\\gpira\\Downloads\\DADOS_COVI\\DADOS_COVI\\PNAD_COVID_072020.csv',
 'C:\\Users\\gpira\\Downloads\\DADOS_COVI\\DADOS_COVI\\PNAD_COVID_082020.csv',
 'C:\\Users\\gpira\\Downloads\\DADOS_COVI\\DADOS_COVI\\PNAD_COVID_092020.csv']

In [334]:
colunas_selecionadas = {
    #Pilar Demográfico
    'UF': 'uf',
    'V1013': 'mes_referencia',
    'A002': 'idade',
    'A003': 'sexo',             
    'A005': 'escolaridade',
    
    # --- Pilar Clínico (Sintomas e Saúde) ---
    'B0011': 'teve_febre',       
    'B0012': 'teve_tosse',
    'B0013': 'teve_dor_garganta',
    'B0014': 'teve_dificuldade_respirar',
    'B0016':'teve_dor_peito',
    'B00111': 'perda_olfato_paladar',
    
    'B002': 'procurou_saude',    
    'B009B': 'resultado_exame',
    
    
    # --- Pilar Comportamental ---
    'B011': 'medida_isolamento',
 
    
    # --- Pilar Econômico ---
    'C001': 'trabalhou_semana_passada',
    'C002': 'afastado_trabalho',
    'C013': 'trabalho_remoto', 
    'D0051': 'auxilio_emergencial',
    'C01011': 'faixa_rendimento',  
    'C007C': 'tipo_trabalho'          }

In [335]:
lista_dfs=[]

for arquivo in lista_arquivos:
    nome_arquivo=os.path.basename(arquivo)
    print(f"🔄 Lendo arquivo: {nome_arquivo}")
    #ler o arquivo com virgula como separador111111111
    print(arquivo)
    df=pd.read_csv(arquivo,
                   sep=',',
                   usecols=lambda column: column in colunas_selecionadas.keys(),
                   dtype=str,
                   low_memory=False)
    print(df.shape)
lista_dfs.append(df)                                  

🔄 Lendo arquivo: PNAD_COVID_072020.csv
C:\Users\gpira\Downloads\DADOS_COVI\DADOS_COVI\PNAD_COVID_072020.csv
(384166, 20)
🔄 Lendo arquivo: PNAD_COVID_082020.csv
C:\Users\gpira\Downloads\DADOS_COVI\DADOS_COVI\PNAD_COVID_082020.csv
(386520, 20)
🔄 Lendo arquivo: PNAD_COVID_092020.csv
C:\Users\gpira\Downloads\DADOS_COVI\DADOS_COVI\PNAD_COVID_092020.csv
(387298, 20)


In [309]:
df=pd.concat(lista_dfs, ignore_index=True)
df.rename(columns=colunas_selecionadas, inplace=True)


In [310]:
df2=pd.read_excel(r"C:\Users\gpira\Downloads\DADOS_COVI\DADOS_COVI\Dicionario_PNAD_COVID_112020_20220621.xls",engine='xlrd',sheet_name='dicionário pnad covid')

In [311]:
#excluir primeira coluna
df2.drop(df2.columns[0],axis=1)
#cabecalho as primeira linha
df2.columns=df2.iloc[0]

df2

,Tamanho,Código\nda\nvariável,Quesito,NaN,Categorias,NaN
0,Tamanho,Código\nda\nvariável,Quesito,NaN,Categorias,NaN
1,NaN,NaN,nº,Descrição,Tipo,Descrição
2,Parte 1 - Identificação e Controle,NaN,NaN,NaN,NaN,NaN
3,4,Ano,NaN,Ano de referência,NaN,NaN
4,2,UF,NaN,Unidade da Federação,11,Rondônia
...,...,...,...,...,...,...
655,2,F0061,F61,Quem respondeu ao questionário?,1,Pessoa moradora
656,NaN,NaN,NaN,NaN,2,Pessoa não moradora
657,NaN,NaN,NaN,NaN,9,Ignorado
658,2,F006,F6,Número de ordem do morador que prestou as info...,1 a 30,Número de ordem do morador


In [312]:
df2['Código\nda\nvariável']=df2['Código\nda\nvariável'].ffill()

In [319]:
df2.columns=df2.columns.str.strip()
colunas=['Tamanho', 'Código_da_variável', 'Quesito', 'Descricao', 'Categorias', 'Valores']
df2.columns=colunas

In [321]:
df2['Descricao']=df2['Descricao'].ffill()

In [325]:
df2['Descricao'].to_csv(os.path.join(caminho_pasta,'dim_variaveis.csv'),index=False)

In [313]:
colunas_selecionadas2=colunas_selecionadas.copy()

In [314]:
colunas_selecionadas['B0014']

'teve_dificuldade_respirar'

In [302]:
colunas_selecionadas2['UF'].to_csv(os.path.join(caminho_pasta,'dim_uf.csv'),index=False)

In [ ]:
nao=['V1013','A002','C01012']
for i in list(colunas_selecionadas2.keys()):
    if i not in nao:
        filtro=df2['Código\nda\nvariável']==i
        colunas_selecionadas2[i]=pd.DataFrame(df2.loc[filtro])
        colunas_selecionadas2[i]=colunas_selecionadas2[i].iloc[:,4:6]
        colunas_selecionadas2[i].columns=['codigo','descricao']
        df[colunas_selecionadas[i]]=df[colunas_selecionadas[i]].map(lambda x: int(x) if pd.notna(x) else x)
        colunas_selecionadas2[i]['codigo']=colunas_selecionadas2[i]['codigo'].map(lambda x: int(x) if pd.notna(x) else x)
        df=df.merge(colunas_selecionadas2[i],left_on=colunas_selecionadas[i],right_on='codigo',how='left')
        df[colunas_selecionadas[i]]=df['descricao']
        df.drop(columns=['codigo','descricao'],inplace=True)
    
    
    
    
    


UF
A003
A005
B0011
B0012
B0014
B00111
B002
B009B
B011
C001
C002
C013
D0051
C01011
C007C


In [ ]:
df.to_csv(os.path.join(caminho_pasta,'dados_tratados_completo.csv'),index=False)


In [290]:
df

,uf,mes_referencia,idade,sexo,escolaridade,teve_febre,teve_tosse,teve_dificuldade_respirar,perda_olfato_paladar,procurou_saude,resultado_exame,medida_isolamento,trabalhou_semana_passada,afastado_trabalho,tipo_trabalho,faixa_rendimento,trabalho_remoto,auxilio_emergencial
0,Rondônia,11,036,Homem,Médio completo,Não,Não,Não,Não,Não aplicável,Não aplicável,"Reduziu o contato com as pessoas, mas continuo...",Sim,Não aplicável,Outro técnico ou profissional de nível médio,801 - 1.600,Não aplicável,Sim
1,Rondônia,11,030,Mulher,Superior completo,Não,Não,Não,Não,Não aplicável,Não aplicável,"Reduziu o contato com as pessoas, mas continuo...",Sim,Não aplicável,"Técnico, profissional da saúde de nível médio",801 - 1.600,Não aplicável,Sim
2,Rondônia,11,013,Homem,Fundamental incompleto,Não,Não,Não,Não,Não aplicável,Não aplicável,Ficou rigorosamente em casa,Não aplicável,Não aplicável,não aplicável,Não aplicável,Não aplicável,Sim
3,Rondônia,11,011,Homem,Fundamental incompleto,Não,Não,Não,Não,Não aplicável,Não aplicável,Ficou rigorosamente em casa,Não aplicável,Não aplicável,não aplicável,Não aplicável,Não aplicável,Sim
4,Rondônia,11,057,Mulher,Fundamental incompleto,Não,Não,Não,Não,Não aplicável,Não aplicável,"Reduziu o contato com as pessoas, mas continuo...",Não,Não,não aplicável,Não aplicável,Não aplicável,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381433,Distrito Federal,11,045,Mulher,Fundamental completa,Não,Não,Não,Não,Não aplicável,Não aplicável,"Reduziu o contato com as pessoas, mas continuo...",Não,Sim,"Faxineiro, auxiliar de limpeza etc. (em empres...",801 - 1.600,Não aplicável,Sim
381434,Distrito Federal,11,022,Mulher,Superior completo,Não,Não,Não,Não,Não aplicável,Não aplicável,"Reduziu o contato com as pessoas, mas continuo...",Não,Sim,"Vendedor a domicílio, representante de vendas,...",801 - 1.600,Não aplicável,Sim
381435,Distrito Federal,11,016,Mulher,Fundamental completa,Não,Não,Não,Não,Não aplicável,Não aplicável,"Reduziu o contato com as pessoas, mas continuo...",Não,Não,não aplicável,Não aplicável,Não aplicável,Sim
381436,Distrito Federal,11,083,Homem,Fundamental incompleto,Não,Não,Não,Não,Não aplicável,Não aplicável,Ficou rigorosamente em casa,Não,Não,não aplicável,Não aplicável,Não aplicável,Não


In [300]:
#criando a dim_mes
lista_meses=[]
for i in range(1,13):
    lista_meses.append({'mes_referencia':i,'mes_nome':pd.to_datetime(f'2020-{i}-01').strftime('%B')})
print(lista_meses)
df_meses=pd.DataFrame(lista_meses)
df_meses.to_csv(os.path.join(caminho_pasta,'dim_mes.csv'),index=False)


[{'mes_referencia': 1, 'mes_nome': 'janeiro'}, {'mes_referencia': 2, 'mes_nome': 'fevereiro'}, {'mes_referencia': 3, 'mes_nome': 'março'}, {'mes_referencia': 4, 'mes_nome': 'abril'}, {'mes_referencia': 5, 'mes_nome': 'maio'}, {'mes_referencia': 6, 'mes_nome': 'junho'}, {'mes_referencia': 7, 'mes_nome': 'julho'}, {'mes_referencia': 8, 'mes_nome': 'agosto'}, {'mes_referencia': 9, 'mes_nome': 'setembro'}, {'mes_referencia': 10, 'mes_nome': 'outubro'}, {'mes_referencia': 11, 'mes_nome': 'novembro'}, {'mes_referencia': 12, 'mes_nome': 'dezembro'}]


In [284]:
#Frequencia Geral de sintomas

Frequencia_sintomas=pd.DataFrame({
'Dificuldade de respirar':df_['teve_dificuldade_respirar'].value_counts(normalize=True)*100,
'Teve Febre':df_['teve_febre'].value_counts(normalize=True)*100,
'Teve Tosse':df_['teve_tosse'].value_counts(normalize=True)*100,
'Perda de olfalto e paladar':df_['perda_olfato_paladar'].value_counts(normalize=True)*100 })

In [286]:
Frequencia_sintomas.reset_index(inplace=True)


In [287]:
Frequencia_sintomas

,index,Dificuldade de respirar,Teve Febre,Teve Tosse,Perda de olfalto e paladar
0,Não,95.750561,92.675764,92.364830,92.744861
1,Sim,3.869407,6.944205,7.237865,6.460529
2,Ignorado,0.293660,0.310935,0.328209,0.708240
3,Não sabe,0.086371,0.069097,0.069097,0.086371


In [277]:
df['teve_tosse'].value_counts(normalize=True)*100


teve_tosse
Não         98.306933
Sim          1.378730
Ignorado     0.259020
Não sabe     0.055317
Name: proportion, dtype: float64

In [278]:
df['perda_olfato_paladar'].value_counts(normalize=True)*100

perda_olfato_paladar
Não         99.200394
Sim          0.384597
Ignorado     0.356545
Não sabe     0.058463
Name: proportion, dtype: float64